In [1]:
from flask import Flask

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['JWT_SECRET_KEY'] = '#Somya2003'

In [2]:
!pip install flask
!pip install flask_sqlalchemy
!pip install flask_jwt_extended
!pip install flask_dance
!pip install werkzeug

In [3]:
from flask import Flask

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['JWT_SECRET_KEY'] = '#Somya2003'

In [4]:
from flask_sqlalchemy import SQLAlchemy

# Initialize the database

db = SQLAlchemy(app)

# Define User and Role models
class Role(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50), unique=True)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(200))
    role_id = db.Column(db.Integer, db.ForeignKey('role.id'))
    
    role = db.relationship('Role', backref='users')

# Create database tables within application context
with app.app_context():
    db.create_all()  # Create database tables if they don't exist

In [5]:
from flask import request, jsonify, redirect, url_for
from werkzeug.security import generate_password_hash, check_password_hash
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity

# Initialize JWT manager

jwt = JWTManager(app)

# User registration endpoint
@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    if not data or not data.get('username') or not data.get('password'):
        return jsonify({"msg": "Username and password are required"}), 400

    hashed_password = generate_password_hash(data['password'], method='pbkdf2:sha256', salt_length=16)
    new_user = User(username=data['username'], password=hashed_password)

    db.session.add(new_user)
    db.session.commit()
    return jsonify({"msg": "User registered successfully"}), 201

# User login endpoint
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    if not data or not data.get('username') or not data.get('password'):
        return jsonify({"msg": "Username and password are required"}), 400

    user = User.query.filter_by(username=data['username']).first()
    if user and check_password_hash(user.password, data['password']):
        access_token = create_access_token(identity={"username": user.username})
        return jsonify(access_token=access_token), 200

    return jsonify({"msg": "Bad username or password"}), 401

In [6]:
from functools import wraps

# Protected route example for admin users
def role_required(role):
    def wrapper(fn):
        @wraps(fn)
        @jwt_required()
        def decorated(*args, **kwargs):
            current_user = get_jwt_identity()
            if current_user['role'] != role:
                return jsonify({"msg": "Access denied"}), 403
            return fn(*args, **kwargs)
        return decorated
    return wrapper

@app.route('/admin', methods=['GET'])
@role_required('Admin')
def admin_route():
    return jsonify({"msg": "Welcome Admin!"}), 200

@app.route('/user', methods=['GET'])
@role_required('User')
def user_route():
    return jsonify({"msg": "Welcome User!"}), 200

In [7]:
from flask_dance.contrib.google import make_google_blueprint, google

# OAuth setup for Google authentication
google_bp = make_google_blueprint(
    client_id='86174551283-oqadmjjkope3qfaff2r7ul0n4ss30k86.apps.googleusercontent.com',
    client_secret='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',
    redirect_to='google_login'
)
app.register_blueprint(google_bp, url_prefix='/google_login')

In [ ]:
if __name__ == "__main__":
    app.run(debug=False)  # Set debug to False

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Nov/2024 18:37:31] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2024 18:37:32] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2024 18:37:32] "GET /ws/ws HTTP/1.1" 404 -


In [ ]:
@app.route('/about')
def about():
    return "This is the About page."

In [ ]:
@app.errorhandler(404)
def not_found(error):
    return "This page does not exist.", 404

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1